In [ ]:
%cd CPMP-ML

/content/CPMP_With_attention/CPMP-ML


In [1]:
import numpy as np
import keras

In [2]:
#@title Carga de modelos directa de github (Experimental)
import requests
import io
url_origin_model = "https://github.com/LosWillyAmigos/CPMP_With_attention/blob/main/models/model_cpmp_origin_5x5_multi.h5"
url_destination_model = "https://github.com/LosWillyAmigos/CPMP_With_attention/blob/main/models/model_cpmp_dest_5x5.h5"

def load_model(url):
  response = requests.get(url)

  if response.status_code == 200:
      return keras.models.load_model(io.BytesIO(response.content))

  print("Error al descargar el modelo.")
  return None

In [3]:
#@title Importado de modelo origin - destino 5x5
origin_model = keras.models.load_model('model_cpmp_origin_5x5_multi.h5')
destination_model = keras.models.load_model('model_cpmp_dest_5x5.h5')

In [67]:
#@title Clase para manejar datos
class CPMP_AI():
  def __init__(self, m1, m2, S, H):
    self.stacks = S
    self.height = H
    self.model_1 = m1
    self.model_2 = m2

  def predict(self, state):
    origin_labels = self.model_1.predict(state)[0]

    origin_labels_categorical = self.label_categorical(origin_labels)

    states = self.states_with_labels(state, origin_labels_categorical)

    destination_labels = self.model_2(states)

    destination_labels_categorical = self.destination_analyzer(destination_labels)

    return zip(origin_labels_categorical, destination_labels_categorical)


  ## ******** OBTENER CATEGORIA DE MODELO ORIGEN ******** ##
  ## Toma el arreglo de predicciones y luego convierte a  ##
  ## etiquetas categoricas.                               ##
  ## input  :                                             ##
  ##  - arr : arreglo de valores entre 0 y 1.             ##
  ##  - sep : cambia el valor de la etiqueta según la se- ##
  ##        paración.                                     ##
  ## output : arreglo de arreglos con las etiquetas con-  ##
  ##        vertidas.                                     ##
  def label_categorical(self, arr = None):
    list_categorical = []
    label = self.analyzer(arr)
    idx_labels = np.where(label == 1)[0]

    for idx in idx_labels:
      list_categorical.append(self.to_labels(idx, self.stacks))
    return np.array(list_categorical)

  def to_labels(self, idx, S):
    arr = np.zeros(S)
    arr[idx] = 1
    return arr

  def destination_analyzer(self, labels):
    destination_list = []
    for _ in labels:
      print(_)
      destination_list.append(self.analyzer(_))

    return np.array(destination_list)
  ## ******** Pasar de vector flotante a binario ******** ##
  ## Recibe un vector de elementos flotantes, este análiza##
  ## Mediante el promedio si se transforma a 1 o 0 el ele-##
  ## mento i-ésimo.                                       ##
  def analyzer(self, arr):
    vector = np.square(arr)
    err = np.mean(vector) + np.std(vector)
    labels = []
    for i in range(arr.shape[0]):
      if vector[i] < err: labels.append(0)
      else: labels.append(1)

    return np.array(labels, dtype='float64')
  ## ******* OBTENER DATOS PARA EL SEGUNDO MODELO ******* ##
  ## Concatenar los labels con el estado inicial para el  ##
  ## segundo modelo.                                      ##
  ## input:                                               ##
  ##      - state : estado actual en forma matricial.     ##
  ##      - labels : lista de etiquetas en forma de array.##
  def states_with_labels(self, state, labels):
    list_states = []

    for label in labels:
      list_states.append(self.concatenate_state(state[0], label))

    return np.array(list_states)

  def concatenate_state(self, state, label):
    new_data = []

    for i in range(state.shape[0]):
       stack = state[i].tolist()
       stack.append(label[i])
       new_data.append(stack)

    return np.stack(new_data)



# Cargar datos

In [31]:
#@title Función para cargar datos

#************** | load_data() | ************#
# El proposito de esta función es cargar    #
# los datos almacenados en un csv para el   #
# entrenamiento de la red neuronal.         #
#                                           #
# Input:                                    #
#     - name: Nombre del archivo con su     #
#             dirección correspondiente     #
#             sin el .csv.                  #
#                                           #
# Output:                                   #
#      Retorna 3 arreglos, el primero       #
#      corresponde a una lista de estados,  #
#      el segundo a una lista de labels     #
#      de stacks de origen y por último una #
#      lista de posibles stacks de destino. #
def load_data(name):
    data = []
    labels_1 = []
    labels_2 = []

    with open(name + '.csv', 'r') as archivo:
        total = int(archivo.readline().split(':')[1])
        line = archivo.readline().split(':')
        size_stacks = int(line[1].split(',')[0])
        size_height = int(line[2])
        archivo.readline()

        for i in range(total):
            matrix = archivo.readline().split(':')[1].split(',')
            matrix = np.array(matrix, dtype= float)
            matrix = np.reshape(matrix, (size_stacks, size_height))

            label_1 = archivo.readline().split(':')[1].split(',')
            label_1 = np.array(label_1, dtype= float)
            label_2 = archivo.readline().split(':')[1].split(',')
            label_2 = np.array(label_2, dtype= float)

            data.append(matrix)
            labels_1.append(label_1)
            labels_2.append(label_2)

            archivo.readline()

    return np.stack(data), np.stack(labels_1), np.stack(labels_2)

In [32]:
data, label_1, label_2 = load_data("data_5x5_SO_Multiple_alpha")

In [68]:
model = CPMP_AI(origin_model,destination_model,5,6)

In [58]:
data[2819], label_1[2819], label_2[2819]

(array([[1.        , 2.        , 2.        , 2.        , 0.93333333,
         0.86666667],
        [0.        , 2.        , 2.        , 0.6       , 0.8       ,
         0.2       ],
        [1.        , 2.        , 2.        , 2.        , 2.        ,
         0.53333333],
        [0.        , 2.        , 0.66666667, 0.2       , 0.33333333,
         0.33333333],
        [0.        , 0.6       , 0.13333333, 0.6       , 0.66666667,
         0.4       ]]),
 array([0., 1., 0., 1., 1.]),
 array([0., 1., 0., 1., 1.]))

In [69]:
labels = model.predict(np.array([data[2819]]))

1/1 [==============================] - 0s 31ms/step
tf.Tensor([2.1842035e-04 1.3219492e-04 9.9951887e-01 4.4355660e-05 8.6173437e-05], shape=(5,), dtype=float32)


In [70]:
for x, y in labels:
  print(x,y)

[0. 0. 0. 0. 1.] [0. 0. 1. 0. 0.]
